# ChargeUp! Battery Geospatial Data Analysis and BSS Location Optimisation (Public) v1.0.0

This notebook provides a framework for the geospatial analysis of e-motorcycle battery location data and the optimisation of battery swap station (BSS) locations, developed as part of the **ChargeUp!** project (2022-2023), funded by **P4G** (https://p4gpartnerships.org/chargeup). 

Author: Cameron Sheehan (Research Associate, Energy Futures Lab, Imperial College London)

## 1. Import required packages

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from keplergl import KeplerGl
import random
from h3 import h3
import h3pandas
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear
import folium
import networkx as nx
import osmnx as ox
import shapely
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
import pulp
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from descartes import PolygonPatch
from rasterstats import zonal_stats
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import normalize
import movingpandas as mpd
import xarray as xr
import hvplot.xarray  # noqa
import hvplot.pandas 
from holoviews import opts
import seaborn as sns
from pytz import common_timezones, all_timezones
import warnings
from IPython.display import display, HTML
import math

display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

plt.rcParams['axes.axisbelow'] = True

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## 2. Choose to run notebook in either demo or production mode

Uncomment the required mode. Demo mode demonstrates the analysis notebook by using a smaller subset of the total dataset so the analyses doesn't take too long to run. 

In [ ]:
notebook_mode = 'demo'
# notebook_mode = 'production'

## 3. Import and pre-process input data

### Open various data files as dataframes and process column data

First, paste full pathname to the data files below, ensure pathname is inside inverted commas, i.e. '/XXX/XXX/XXX/battery-bms.csv'

In [ ]:
# Set data file pathnames
pathname_battery_gps = ''
pathname_battery_swaps = ''
pathname_battery_bms = ''
pathname_BSS_loc = ''
pathname_population = ''

Next, input the column headings for the required data fields:

In [ ]:
# For GPS dataset
gps_timestamp_col = ''
batt_id_col = ''
gps_lat_col = ''
gps_lng_col = ''

# For BSS locations dataset
BSS_lat_col = ''
BSS_lng_col = ''
BSS_id_col = '';
BSS_loc_name_col = ''

Read data from csv/parquet files and create pandas dataframes: <br>

Note: Uncomment the correct option depending on file types csv/parquet

In [ ]:
# Read data from csv file and create a pandas dataframe
df_raw = pd.read_csv(pathname_battery_gps, header=0) 
# df = pd.read_parquet(pathname_battery_gps) 
df = pd.DataFrame()

df_existing_BSS_loc_raw = pd.read_csv(pathname_BSS_loc, header=0)
# df_existing_BSS_loc = pd.read_parquet(pathname_BSS_loc)
df_existing_BSS_loc = pd.DataFrame()

If the Notebook is being run in 'demo' mode, create a smaller subset of the raw GPS data to be used in the analyis. Change the number to suit your dataset.

In [ ]:
if notebook_mode == 'demo':
    df_raw = df_raw.iloc[0:100000]

In [ ]:
df_raw.info()

Assign data columns to correct dataframe column headings for required data fields:

In [ ]:
df[['gps_timestamp','batt_id','gps_lat','gps_lng']] = df_raw[[gps_timestamp_col,batt_id_col,gps_lat_col,gps_lng_col]]

df_existing_BSS_loc[['BSS_id','BSS_loc_name','BSS_lat','BSS_lng']] = df_existing_BSS_loc_raw[[BSS_id_col,BSS_loc_name_col,BSS_lat_col,BSS_lng_col]]

Process data types and required timezones:

In [ ]:
# Process battery_gps data
df['batt_id'] = df['batt_id'].astype("string")
df['gps_time'] = pd.to_datetime(df['gps_timestamp'], unit='s', utc=True).dt.tz_convert('Africa/Nairobi')
df = df.set_index('gps_time')

# Create geodataframes from dataframes. Ensure co-ordinate reference system (crs) is correct. 
# Note EPSG 4326 is WGS 84 (https://epsg.io/4326)
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.gps_lng, df.gps_lat, crs="EPSG:4326"), crs="EPSG:4326")

# Process existing_BSS_loc data
gdf_existing_BSS_loc = gpd.GeoDataFrame(
    df_existing_BSS_loc, geometry=gpd.points_from_xy(df_existing_BSS_loc.BSS_lng, df_existing_BSS_loc.BSS_lat, crs="EPSG:4326"), crs="EPSG:4326")

# Display first 5 lines of geodataframe to inspect data
gdf.head()

In [ ]:
# Display first 5 lines of geodataframe to inspect data
gdf_existing_BSS_loc.head()

In [ ]:
# Check data types of each column. Ensure 'geometry' column is 'geometry' data type.
gdf.info()

Determine the total number of unique batteries in the GPS dataset

In [ ]:
n_batteries = len(gdf.batt_id.unique())
print("There are a total of ", n_batteries, " unique batteries, and ", len(gdf), " data entries in this BMS dataset")

#### Import Nairobi boundaries and other relevant map data from OpenStreetMaps using OSMNX package

In [ ]:
# Import Nairobi boundary and add a small buffer to allow trips that were just outside of the border to be included in the analysis
NBO_dist_bndry = gpd.GeoDataFrame(geometry = ox.geocode_to_gdf('Nairobi').geometry.buffer(0.02));
# Import Nairobi National Park boundary as an area to be removed from the anlaysis region
NBO_nat_park_bndry = gpd.GeoDataFrame(geometry = ox.geocode_to_gdf('Nairobi national park').geometry)
# Removed Nairobi National Park from the anlaysis region
NBO_dist_bndry['geometry'] = NBO_dist_bndry.difference(NBO_nat_park_bndry)

#### Display analysis boundary on Kepler gl map

In [ ]:
m_bound = KeplerGl(height=800)
# m.add_data(gdf.copy(), 'All GPS Points') # uncomment to add GPS points to map - then add new layer in Kepler gl
m_bound.add_data(NBO_dist_bndry.copy(), 'Nairobi Boundary (excluding national parks)')
%run 'Kepler configs/map_config_bound.py'
m_bound.config = config

In [ ]:
m_bound

Save map configuration

In [ ]:
# with open('Kepler configs/map_config_bound.py', 'w') as f:
#    f.write('config = {}'.format(m_bound.config))

## 4. Create H3 hexagon dataframe for Nairobi

#### Set H3 resolution and input edge length for analysis - details: https://h3geo.org/docs/core-library/restable

In [ ]:
# Set H3 resolution
res = 9
# Find and input the hexagon edge length for the set H3 resolution from https://h3geo.org/docs/core-library/restable/#edge-lengths 
h3_hex_edge_length = 0.174375668 # km for res = 9

# Calculate other hex dimensions based on edge length 
h3_hex_across_corners = 2*h3_hex_edge_length
h3_hex_across_flats = math.sqrt(3)*h3_hex_edge_length
# Set number (k) of hexagon rings for analysis (max of 5)
k_rings = 5

print("The following parameters have been set for the H3 hexagons: \n H3 Resolution:", res, 
      "\n H3 Hexagon distance across flats / between centres: ", np.round(h3_hex_across_flats*1000, decimals=1), " m",
     "\n Number of hexagon 'rings' around each hexagon to use for analysis: ", k_rings)

#### Fill Nairobi boundary with H3 hexagons

In [ ]:
NBO_h3 = NBO_dist_bndry.h3.polyfill(res, explode=True).reset_index(drop=True).drop(columns=['geometry'])

#### Determine which H3 hexagons the existing BSSs are located in

In [ ]:
gdf_existing_BSS_h3 = gdf_existing_BSS_loc.h3.geo_to_h3(res)
# Create a list of the unique H3 hex's where existing BSSs are located
existing_BSS_h3_list = gdf_existing_BSS_h3.index.unique().to_list()
# Create geometry of H3 hexagons that the existing BSSs are located in
gdf_existing_BSS_h3_hex = gdf_existing_BSS_h3.h3.h3_to_geo_boundary().reset_index(drop=True)
# Display the list of H3 indexes where BSS are located. 
# Note that the list may be shorter than the number of existing BSSs if some BSSs are located in the same H3 hex.
existing_BSS_h3_list

## 5. Analyse movemement/trip data

#### Create a movingpandas function to analyse gps movemement data:

Function adapted from (Vallejo, B.R., 2021): https://towardsdatascience.com/stop-detection-in-blue-whales-gps-tracking-movingpandas-0-6-55a4b893a592

In [ ]:
def get_stop_point_traj_collection(moving_df, traject_id_column, minutes, searchradio):
    ''' Parameters
    - moving_df <geodataframe>: geodataframe of moving data formated with DateTime index
    - traject_id_column <string>: column with tag id of individuals
    - minutes <number>: minutes considered as minimum for the stop detection
    - searchradio <number>: radio in meters considered for the stop detection
    
    Return
    - <geodataframe>: stops as point with time duration and tag id of individuals
    '''
    warnings.filterwarnings('ignore')
    warnings.simplefilter('ignore')
    
    all_stop_points = gpd.GeoDataFrame()
    
    # create a traj collection with movingpandas
    traj_collection = mpd.TrajectoryCollection(moving_df, traject_id_column)
    
    for i in range(len(traj_collection)):

        # create a stop detector
        detector = mpd.TrajectoryStopDetector(traj_collection.trajectories[i])
        
        # stop points
        stop_points = detector.get_stop_points(min_duration=timedelta(minutes=minutes), max_diameter=searchradio)
        
        # add duration to table
        stop_points['duration_m'] = (stop_points['end_time']-stop_points['start_time']).dt.total_seconds()/60
        
        # add ID
        stop_points['tag_id'] = [tag.split('_')[0] for tag in stop_points.index.to_list()]
        
        all_stop_points= all_stop_points.append(stop_points)
        
    return all_stop_points, traj_collection

### Determine all trajectories and stop points

#### Set stop detection parameters

In [ ]:
# Set stop search diameter (m) for detecting stops
stop_diameter = 200;
# Set minimum stop duration (min) for detecting stops
stop_time_min = 5
# Set minimum length of trajectories (m) to keep
traj_length_min = 500

#### Apply function to data to determine all trajectories and stop points

Note that this may take a few minutes to complete depending on the number of GPS points to analyse.

In [ ]:
%%time
em_stops, em_traj = get_stop_point_traj_collection(gdf, 'batt_id', stop_time_min, stop_diameter) 

#### Split trajectories at stopping points

Note that this may take a few minutes to complete depending on the number of trajectories to analyse.

In [ ]:
%%time
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Split trajectories based on stopping times, 
# Stops are detected where the GPS location stays within a set maximum diameter for a longer than a set minimum duration of time 
# Ignore trajectories with a length smaller than the set minimum length.
split = mpd.StopSplitter(em_traj).split(max_diameter=stop_diameter, min_duration=timedelta(minutes=stop_time_min), min_length=traj_length_min)
# Create geodataframe from of split trajectories
gdf_traj = split.to_traj_gdf()
# Convert trip length from meters to km
gdf_traj['length_km']=gdf_traj['length']/1000

# Display resulting geodataframe
gdf_traj.head()

### Determine various trip statistics

#### Trip distances and number of trips

In [ ]:
gdf_traj['length_km'].describe()

In [ ]:
# Determine total trip length in km
total_trip_distance = gdf_traj['length_km'].sum();
total_no_of_trips = len(gdf_traj);
print("The total number of trips and total distance of all trips detected in this dataset is: \n ", total_no_of_trips," trips \n", np.round(total_trip_distance), " km ")

In [ ]:
# Determine average (mean) trip length in km
mean_trip_distance = gdf_traj['length_km'].mean();
print("The average trip distance of all trips detected in this dataset is: \n ", np.round(mean_trip_distance, decimals=2), " km ")

#### Histogram of trip distances

In [ ]:
ax = gdf_traj['length_km'].plot.hist(bins=np.arange(0,30,1), xlim=[-0.5,30], alpha=1, color="blue", edgecolor = "white", figsize=(8,4));# ylim=[0,1000], 
# ax.set_title('Histogram of trip distances');
ax.set_title('');
ax.set_xlabel("Trip distance (km)");
ax.set_ylabel("Number of trips");

In [ ]:
# ax.figure.savefig("images/Histogram of trip distances.png", bbox_inches='tight')

#### Determine trip average speeds

In [ ]:
gdf_traj['trip_duration'] = (gdf_traj['end_t'] - gdf_traj['start_t']).dt.total_seconds()/60
gdf_traj['speed_avg'] = gdf_traj['length_km']/(gdf_traj['trip_duration']/60)
gdf_traj.head()

#### Histogram of trip average speeds

In [ ]:
ax = gdf_traj['speed_avg'].plot.hist(bins=np.arange(0,75,5), alpha=1, color="blue", edgecolor = "white", figsize=(8,4)); #bins=np.arange(0,30,1), xlim=[-0.5,30], ylim=[0,1000],
# ax.set_title('Histogram of trip average speeds');
ax.set_title('');
ax.set_xlabel("Trip average speed (km/h)");
ax.set_ylabel("Number of trips");

In [ ]:
# ax.figure.savefig("images/Histogram of trip average speeds.png", bbox_inches='tight')

In [ ]:
speed_avg = gdf_traj['speed_avg'].mean()
speed_median = gdf_traj['speed_avg'].median()
print("The average trip speed of all trips detected in this dataset is: \n ", np.round(speed_avg, decimals=1), " km/h   \n",
     "The median trip speed of all trips detected in this dataset is: \n ", np.round(speed_median, decimals=1), " km/h")

#### Investigate start times of trips

In [ ]:
# Add time increments for various analyses (note: this method ignores seconds when categorising time increments)

gdf_traj['hr_inc'] = gdf_traj['start_t'].dt.hour + np.ceil(gdf_traj['start_t'].dt.minute/60)
gdf_traj.loc[gdf_traj['hr_inc']==0, 'hr_inc'] = 24
gdf_traj['hr_inc'] = gdf_traj['hr_inc'].astype('int')

gdf_traj['30_mins_inc'] = gdf_traj['start_t'].dt.hour + np.ceil(gdf_traj['start_t'].dt.minute/30)*30/60
gdf_traj.loc[gdf_traj['30_mins_inc']==0, '30_mins_inc'] = 24

gdf_traj['15_mins_inc'] = gdf_traj['start_t'].dt.hour + np.ceil(gdf_traj['start_t'].dt.minute/15)*15/60
gdf_traj.loc[gdf_traj['15_mins_inc']==0, '15_mins_inc'] = 24

gdf_traj.head()

#### Histogram of trip start times (30 mins)

In [ ]:
gdf_traj_plot = gdf_traj
# Reduce the increment by 0.01 to ensure that the manually created increments are binned correctly 
# Histogram bins are exclsuive of the right side bin edge, so a value fo 1.5 would be incorrectly binned between 1.5-2.0, instead of 1.0-1.5
gdf_traj_plot['30_mins_inc'] = gdf_traj_plot['30_mins_inc']-0.01 

ax = gdf_traj_plot['30_mins_inc'].plot.hist(bins=np.arange(0,24.5,0.5),xticks=np.arange(0,25,1), alpha=1, color="blue", edgecolor = "white", figsize=(8,3)); #bins=np.arange(0,30,1), xlim=[-0.5,30], ylim=[0,1000], [0,3,6,9,12,15,18,21,24]
# ax.set_title('Histogram of trip start times');
ax.set_title('');
ax.set_xlabel("Trip start time period (hours)");
ax.set_ylabel("Number of trips");

In [ ]:
# ax.figure.savefig("images/Histogram of trip start times (30 mins).png", bbox_inches='tight')

#### Histogram of trip start times (hours)

In [ ]:
# Reduce the increment by 0.01 to ensure that the manually created increments are binned correctly 
# Histogram bins are exclsuive of the right side bin edge, so a value fo 1.5 would be incorrectly binned between 1.5-2.0, instead of 1.0-1.5
gdf_traj_plot['hr_inc'] = gdf_traj_plot['hr_inc']-0.01 

ax = gdf_traj_plot['hr_inc'].plot.hist(bins=np.arange(0,25,1),xticks=np.arange(0,25,1), alpha=1, color="blue", edgecolor = "white", figsize=(8,3)); #bins=np.arange(0,30,1), xlim=[-0.5,30], ylim=[0,1000], [0,3,6,9,12,15,18,21,24]
# ax.set_title('Histogram of trip start times');
ax.set_title('');
ax.set_xlabel("Trip start time period (hours)");
ax.set_ylabel("Number of trips");

In [ ]:
# ax.figure.savefig("images/Histogram of trip start times (hours).png", bbox_inches='tight')

## 6. Aggregate trip data using H3 hexagons

#### Remove datetime format from gdf and stops indexes

In [ ]:
gdf = gdf.reset_index(drop=True)
em_stops= em_stops.reset_index(drop=True)

#### Add longitude and latitude columns for stop points

In [ ]:
em_stops['Lng'] = em_stops['geometry'].x
em_stops['Lat'] = em_stops['geometry'].y

#### Determine H3 hex indexes of all GPS data points and stopping locations

In [ ]:
# Create a column for the H3 hex indices
hex_col = 'hex'+str(res)

# find hexs containing the points
gdf[hex_col] = gdf.apply(lambda x: h3.geo_to_h3(x.gps_lat,x.gps_lng,res),1)
em_stops[hex_col] = em_stops.apply(lambda x: h3.geo_to_h3(x.Lat,x.Lng,res),1)

# aggregate the points
gdf_g = gdf.groupby(hex_col).size().to_frame('count_val').reset_index()
gdf_g = gdf_g.set_index(hex_col).h3.h3_to_geo_boundary()
em_stops_g = em_stops.groupby(hex_col).size().to_frame('count_val').reset_index()
em_stops = em_stops.drop(columns=[hex_col])

#### Create hexagon geometries from H3 index values for analysis region

In [ ]:
NBO_h3 = NBO_h3.set_index('h3_polyfill').h3.h3_to_geo_boundary()

#### Determine which H3 hex geometries are intersected with trips, then aggregate trip counts

In [ ]:
hex_intersects = gpd.sjoin(NBO_h3, gdf_traj, how="inner", predicate='intersects')
hex_intersects['const']=1

# Group all H3 intersection hexes by their index values and count them
hex_intersects_g = hex_intersects.groupby(['h3_polyfill']).size().to_frame('count').reset_index() 

# Display resulting dataframe with H3 indexes and trip counts
hex_intersects_g.head()

#### Merge the H3 hexes that have trip intersections with the rest of the Nairobi H3 hexes

In [ ]:
hex_intersects_merged = NBO_h3.merge(hex_intersects_g, how='left', on='h3_polyfill').fillna(0)

# Display merged H3 hex dataframe with trip counts for all hexes
hex_intersects_merged.head()

### Visualise H3 Hex trip heatmap with trip trajectories

#### *Important Data Protection Note: This map may contain potentially sensitive location data of individual trips and stops. Depending on the contents of this map, it should not be published publicly without the explicit consent of all individuals whose battery GPS data was included in the analysed dataset.*

In [ ]:
m_heat = KeplerGl(height=800)
hex_intersects_merged_m = hex_intersects_merged[['geometry','count']]
m_heat.add_data(hex_intersects_merged_m.copy(), 'Merged Hex aggregated')
# m_heat.add_data(gdf_traj[['traj_id','geometry','length_km']].copy(), 'GPS Trip Trajectories - split') # Uncomment to include individual trips - Add as new Kepler gl layer
# m_heat.add_data(em_stops.copy(), 'Stop points') # Uncomment to include stop points  - Add as new Kepler gl layer
m_heat.add_data(NBO_h3.copy(),'All H3 hexes')
# load the config
%run 'Kepler configs/map_config_heat.py'
m_heat.config = config

In [ ]:
m_heat

Save map configuration

In [ ]:
# with open('Kepler configs/map_config_heat.py', 'w') as f:
#    f.write('config = {}'.format(m_heat.config))

### Create groups of "neighbouring" hexagons to be used in optimisation

#### Create lists of H3 indices for location hexagons and "extra" hex_ring hexagons

In [ ]:
# Set index of dataframe to the h3 index values created during the H3 polyfill process
NBO_h3 = hex_intersects_merged.set_index('h3_polyfill')
# Create a list of "buffer" hexagons to be used later 
NBO_h3_buff_list = NBO_h3
# Use the H3 k_ring method to create a total of "k" rings
NBO_h3_buff_list = NBO_h3_buff_list.h3.k_ring(k_rings, explode=True)

In [ ]:
# Drop all rows where the h3_k_ring index values are repeated
NBO_h3_buff_list = NBO_h3_buff_list.drop_duplicates(subset=['h3_k_ring'], keep='first')
# Create list from the h3_k_ring index values
NBO_h3_buff_list = NBO_h3_buff_list.h3_k_ring.to_list()
# Create list of h3 index values that are in the analysis region
NBO_h3_analysis_list = NBO_h3.index.to_list()

#### Remove analysis region hex indices from "buffer" hex ring list, i.e. leave ONLY extra neighbor hexagons that are not part of the analysis set of hexagons.

In [ ]:
for i in NBO_h3_buff_list[:]:
        if i in NBO_h3_analysis_list:
            NBO_h3_buff_list.remove(i)

#### Add H3 hex_ring column to Nairobi locations H3 dataframe, to indicate hex neighbours for each index

In [ ]:
for k in range (1, k_rings+1):
    NBO_h3 = NBO_h3.h3.hex_ring(k)
    NBO_h3.rename(columns={'h3_hex_ring':'h3_hex_ring_'+str(k)}, inplace = True)
    
NBO_h3.head()

#### Remove neighbour indices from hex_ring column that are outdside of the analysis boundary set (i.e. remove "extra" hexagons)

Note: This step may take a few mins to complete.

In [ ]:
remove_indices = NBO_h3_buff_list
for k in range (1, k_rings+1):
    for i in range(len(NBO_h3['h3_hex_ring_'+str(k)])):
        for r in remove_indices:
            if r in NBO_h3['h3_hex_ring_'+str(k)][i]:
                NBO_h3['h3_hex_ring_'+str(k)][i].remove(r)

### Add population data from GeoTiff to H3 hexes

In [ ]:
NBO_h3['pop'] = NBO_h3['geometry'].apply(lambda x: zonal_stats(x, pathname_population, stats=['sum'])[0]['sum']).fillna(0)
NBO_h3 = NBO_h3.h3.cell_area()
NBO_h3['pop_dens'] = NBO_h3['pop']/NBO_h3['h3_cell_area']

## 7. Process the data for optimisation

In [ ]:
# Set all counts below the set minimum count limit to zero
min_trip_count = 10
NBO_h3.loc[NBO_h3['count'] <= min_trip_count, 'count'] = 0 

In [ ]:
# Scale data to values between zero and one [0,1]
min_max_scaler = preprocessing.MinMaxScaler()

NBO_h3['pop_scaled'] = min_max_scaler.fit_transform(NBO_h3['pop'].values.reshape(-1, 1)) #returns a numpy array
NBO_h3['trips_scaled'] = min_max_scaler.fit_transform(NBO_h3['count'].values.reshape(-1, 1)) #returns a numpy array

# Display dataframe
NBO_h3.head()

#### Set score weighting values (all weights must add up to 1.0)

pop_w = Population score weighting <br>
trips_w = Trip volume score weighting

In [ ]:
trips_w = 1.0 # Set weight value of trips between 0 and 1 (0 - 100%)

pop_w = 1.0 - trips_w # Weight value of population data calculated as remaining weight percentage
score_w = {'pop_w':pop_w, 'trips_w':trips_w}

In [ ]:
# Determine combined score based on weightings
NBO_h3['score_comb'] = NBO_h3['pop_scaled']*score_w['pop_w'] + NBO_h3['trips_scaled']*score_w['trips_w']
NBO_h3.head()

### Visualise H3 trip counts and population scores using Kepler.gl

In [ ]:
m_trips_pop = KeplerGl(height=800)
m_trips_pop.add_data(NBO_h3.copy(), 'Trip + Pop Data')
# load the config
%run 'Kepler configs/map_config_trips_pop.py'
m_trips_pop.config = config

In [ ]:
m_trips_pop

Save map config to a file

In [ ]:
# with open('Kepler configs/map_config_trips_pop.py', 'w') as f:
#    f.write('config = {}'.format(m_trips_pop.config))

## 8. Setup Mixed-Integer Linear Programming (MILP) Optimisation using PuLP

### Set optimisation inputs for each stage:

#### Stage 1 Optimisation parameters

In [ ]:
R_01 = 10 # Number of BSSs to be built
R_ex_01 = len(existing_BSS_h3_list) # Number of existing BSS sites
w_01 = [1.0,0.5,0.3,0.2,0.1,0.05] # Coverage weightings for each ring of hexagons, first value is the centre hex
M_01 = 1.0
df_01 = NBO_h3
df_01['coverage'] = 0
n_rows_01 = len(df_01)

In [ ]:
R_ex_01

#### Stage 2 Optimisation parameters

In [ ]:
R_02 = 10 # Number of BSSs to be built
R_ex_02 = R_01 + R_ex_01 # Number of existing BSS sites
w_02 = [1.0,0.5,0.3,0.2,0,0] # Coverage weightings for each ring of hexagons, first value is the centre hex
M_02 = 1.0
df_02 = NBO_h3
df_02['coverage'] = 0
n_rows_02 = len(df_02)

In [ ]:
R_ex_02

#### Stage 3 Optimisation parameters

In [ ]:
R_03 = 10 # Number of BSSs to be built
R_ex_03 = R_02 + R_ex_02 # Number of existing BSS sites
w_03 = [1.0,0.5,0,0,0,0] # Coverage weightings for each ring of hexagons, first value is the centre hex
M_03 = 1.0
df_03 = NBO_h3
df_03['coverage'] = 0
n_rows_03 = len(df_03)

In [ ]:
R_ex_03

### Stage 1 Optimisation

In [ ]:
R = R_01 # Number of BSSs to be built
R_ex = R_ex_01  # Number of existing BSS sites
existing_BSS_h3_list_updated = existing_BSS_h3_list # Existing BSS H3 hex list
w = w_01  # Coverage weightings for each ring of hexagons, first value is the centre hex
M = M_01
df = df_01
n_rows = n_rows_01 

#### Declare problem

In [ ]:
prob = LpProblem("max_bss_coverage", LpMaximize)

#### Define the decision variables

In [ ]:
df['decision'] = [LpVariable(name="y"+str(i), cat='Binary') for i in range(n_rows)]

#### Add constraints

In [ ]:
prob += lpSum([df['decision']]) <= R + R_ex   

In [ ]:
for j in range(R_ex):
    prob += df.loc[existing_BSS_h3_list_updated[j]].decision == 1
    
    # Note: If there is an error in this step, it may be because the existing station does not 
    # lie within the boundary selected for this analysis. Either increase the size of the boundary 
    # or remove the station causing the issue from the list.

In [ ]:
for i in range(n_rows):
    prob +=  w[0]*df['decision'][i] + lpSum(lpSum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].decision) for k in range(1,k_rings+1))  <= M

#### Define objective function

In [ ]:
prob += lpSum([df['score_comb'][i]*(w[0]*df['decision'][i] + lpSum(lpSum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].decision) for k in range(1,k_rings+1))) for i in range(n_rows)])

#### Solve problem

In [ ]:
prob.solve();

In [ ]:
pulp.LpStatus[prob.status]

#### Create dataframe of Stage 1 Optimisation results

In [ ]:
df['opt_site'] = [df['decision'][i].varValue for i in range(n_rows)]  
df['coverage'] = [w[0]*df['opt_site'][i] + sum(sum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].opt_site) for k in range(1,k_rings+1)) for i in range(n_rows)]
df['coverage_score'] = df['score_comb']*df['coverage']

df_results_01 = df[['opt_site','coverage_score','coverage', 'count', 'pop', 'score_comb', 'geometry']]
df_opt_sites_01 = df_results_01[df_results_01['opt_site']==1]
gdf_opt_sites_01 = df_opt_sites_01.h3.h3_to_geo_boundary().reset_index(drop=True)

In [ ]:
len(df_opt_sites_01)

### Visualise optimal site results and existing BSS sites

In [ ]:
m_stage1 = KeplerGl(height=800)
m_stage1.add_data(df_results_01.copy(), 'Results')
m_stage1.add_data(gdf_opt_sites_01.copy(), 'Optimal BSS sites')
m_stage1.add_data(gdf_existing_BSS_loc.copy(), 'Existing BSS sites')
m_stage1.add_data(gdf_existing_BSS_h3_hex.copy(), 'Existing BSS site H3 hexes')
%run 'Kepler configs/map_config_stage1.py'
m_stage1.config = config

In [ ]:
m_stage1

In [ ]:
# with open('Kepler configs/map_config_stage1.py', 'w') as f:
#    f.write('config = {}'.format(m_stage1.config))

### Stage 2 Optimisation

In [ ]:
R = R_02 # Number of BSSs to be built
R_ex = R_ex_02  # Number of existing BSS sites
w = w_02  # Coverage weightings for each ring of hexagons, first value is the centre hex
M = M_02
df = df_02
n_rows = n_rows_02 
existing_BSS_h3_list_updated = df_opt_sites_01.index.unique().to_list() # Existing BSS H3 hex list

#### Declare problem

In [ ]:
prob = LpProblem("max_bss_coverage", LpMaximize)

#### Define the decision variables

In [ ]:
df['decision'] = [LpVariable(name="y"+str(i), cat='Binary') for i in range(n_rows)]

#### Add constraints

In [ ]:
prob += lpSum([df['decision']]) <= R + R_ex   

In [ ]:
for j in range(R_ex):
    prob += df.loc[existing_BSS_h3_list_updated[j]].decision == 1
    
    # Note: If there is an error in this step, it may be because the existing station does not 
    # lie within the boundary selected for this analysis. Either increase the size of the boundary 
    # or remove the station causing the issue from the list.

In [ ]:
for i in range(n_rows):
    prob +=  w[0]*df['decision'][i] + lpSum(lpSum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].decision) for k in range(1,k_rings+1))  <= M

#### Define objective function

In [ ]:
prob += lpSum([df['score_comb'][i]*(w[0]*df['decision'][i] + lpSum(lpSum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].decision) for k in range(1,k_rings+1))) for i in range(n_rows)])

#### Solve problem

In [ ]:
prob.solve();

In [ ]:
pulp.LpStatus[prob.status]

#### Create dataframe of Stage 2 Optimisation results

In [ ]:
df['opt_site'] = [df['decision'][i].varValue for i in range(n_rows)]  
df['coverage'] = [w[0]*df['opt_site'][i] + sum(sum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].opt_site) for k in range(1,k_rings+1)) for i in range(n_rows)]
df['coverage_score'] = df['score_comb']*df['coverage']

df_results_02 = df[['opt_site','coverage_score','coverage', 'count', 'pop', 'score_comb', 'geometry']]
df_opt_sites_02 = df_results_02[df_results_02['opt_site']==1]
gdf_opt_sites_02 = df_opt_sites_02.h3.h3_to_geo_boundary().reset_index(drop=True)

In [ ]:
len(df_opt_sites_02)

### Visualise optimal site results and existing BSS sites

In [ ]:
m_stage2 = KeplerGl(height=800)
m_stage2.add_data(df_results_02.copy(), 'Results')
m_stage2.add_data(gdf_opt_sites_02.copy(), 'Stage 2 Optimal BSS sites')
m_stage2.add_data(gdf_opt_sites_01.copy(), 'Stage 1 Optimal BSS sites')
m_stage2.add_data(gdf_existing_BSS_loc.copy(), 'Existing BSS sites')
m_stage2.add_data(gdf_existing_BSS_h3_hex.copy(), 'Existing BSS site H3 hexes')
%run 'Kepler configs/map_config_stage2.py'
m_stage2.config = config

In [ ]:
m_stage2

In [ ]:
# with open('Kepler configs/map_config_stage2.py', 'w') as f:
#    f.write('config = {}'.format(m_stage2.config))

### Stage 3 Optimisation

In [ ]:
R = R_03 # Number of BSSs to be built
R_ex = R_ex_03  # Number of existing BSS sites
w = w_03  # Coverage weightings for each ring of hexagons, first value is the centre hex
M = M_03
df = df_03
n_rows = n_rows_03 
existing_BSS_h3_list_updated = df_opt_sites_02.index.unique().to_list() # Existing BSS H3 hex list

#### Declare problem

In [ ]:
prob = LpProblem("max_bss_coverage", LpMaximize)

#### Define the decision variables

In [ ]:
df['decision'] = [LpVariable(name="y"+str(i), cat='Binary') for i in range(n_rows)]

#### Add constraints

In [ ]:
prob += lpSum([df['decision']]) <= R + R_ex   

In [ ]:
for j in range(R_ex):
    prob += df.loc[existing_BSS_h3_list_updated[j]].decision == 1
    
    # Note: If there is an error in this step, it may be because the existing station does not 
    # lie within the boundary selected for this analysis. Either increase the size of the boundary 
    # or remove the station causing the issue from the list.

In [ ]:
for i in range(n_rows):
    prob +=  w[0]*df['decision'][i] + lpSum(lpSum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].decision) for k in range(1,k_rings+1))  <= M                                                           

#### Define objective function

In [ ]:
prob += lpSum([df['score_comb'][i]*(w[0]*df['decision'][i] + lpSum(lpSum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].decision) for k in range(1,k_rings+1))) for i in range(n_rows)])

#### Solve problem

In [ ]:
prob.solve();

In [ ]:
pulp.LpStatus[prob.status]

#### Create dataframe of Stage 3 Optimisation results

In [ ]:
df['opt_site'] = [df['decision'][i].varValue for i in range(n_rows)]  
df['coverage'] = [w[0]*df['opt_site'][i] + sum(sum(w[k]*df.loc[df['h3_hex_ring_'+str(k)][i]].opt_site) for k in range(1,k_rings+1)) for i in range(n_rows)]
df['coverage_score'] = df['score_comb']*df['coverage']

df_results_03 = df[['opt_site','coverage_score','coverage', 'count', 'pop', 'score_comb', 'geometry']]
df_opt_sites_03 = df_results_03[df_results_03['opt_site']==1]
gdf_opt_sites_03 = df_opt_sites_03.h3.h3_to_geo_boundary().reset_index(drop=True)

### Visualise optimal site results and existing BSS sites

In [ ]:
m_stage3 = KeplerGl(height=800)
m_stage3.add_data(df_results_03.copy(), 'Results')
m_stage3.add_data(gdf_opt_sites_01.copy(), 'Stage 1 Optimal BSS sites')
m_stage3.add_data(gdf_opt_sites_02.copy(), 'Stage 2 Optimal BSS sites')
m_stage3.add_data(gdf_opt_sites_03.copy(), 'Stage 3 Optimal BSS sites')
m_stage3.add_data(gdf_existing_BSS_loc.copy(), 'Existing BSS sites')
m_stage3.add_data(gdf_existing_BSS_h3_hex.copy(), 'Existing BSS site H3 hexes')
m_stage3.add_data(NBO_h3['geometry'].reset_index().copy(),'All H3 hexes')
%run 'Kepler configs/map_config_stage3.py'
m_stage3.config = config

In [ ]:
m_stage3

In [ ]:
# with open('Kepler configs/map_config_stage3.py', 'w') as f:
#    f.write('config = {}'.format(m_stage3.config))